## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Atuona,PF,-9.8000,-139.0333,78.85,74,100,16.40,overcast clouds
1,Nikolskoye,RU,59.7035,30.7861,31.87,97,100,4.32,overcast clouds
2,Longyearbyen,SJ,78.2186,15.6401,4.84,92,0,9.22,clear sky
3,Saskylakh,RU,71.9167,114.0833,-12.64,91,100,3.74,overcast clouds
4,Kapaa,US,22.0752,-159.3190,80.58,74,40,10.36,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Atuona,PF,-9.8000,-139.0333,78.85,74,100,16.40,overcast clouds
4,Kapaa,US,22.0752,-159.3190,80.58,74,40,10.36,scattered clouds
5,Beibei,CN,29.8256,106.4339,81.07,51,70,2.06,broken clouds
17,Busselton,AU,-33.6500,115.3333,78.31,64,0,13.65,clear sky
26,Rikitea,PF,-23.1203,-134.9692,78.84,67,1,3.24,clear sky
27,Vaini,TO,-21.2000,-175.2000,86.16,84,75,9.22,broken clouds
33,Hilo,US,19.7297,-155.0900,80.33,69,0,6.91,clear sky
37,Lorengau,PG,-2.0226,147.2712,85.05,68,31,4.25,scattered clouds
38,Makakilo City,US,21.3469,-158.0858,82.72,51,0,13.80,clear sky
39,Paciran,ID,-6.8767,112.3761,84.42,71,100,3.09,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   164
Country                164
Lat                    164
Lng                    164
Max Temp               164
Humidity               164
Cloudiness             164
Wind Speed             164
Current Description    164
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Atuona,PF,-9.8000,-139.0333,78.85,74,100,16.40,overcast clouds
4,Kapaa,US,22.0752,-159.3190,80.58,74,40,10.36,scattered clouds
5,Beibei,CN,29.8256,106.4339,81.07,51,70,2.06,broken clouds
17,Busselton,AU,-33.6500,115.3333,78.31,64,0,13.65,clear sky
26,Rikitea,PF,-23.1203,-134.9692,78.84,67,1,3.24,clear sky
...,...,...,...,...,...,...,...,...,...
648,Bosaso,SO,11.2842,49.1816,75.56,70,89,5.70,overcast clouds
650,Byron Bay,AU,-28.6500,153.6167,77.94,56,90,11.68,overcast clouds
651,Wajir,KE,1.7471,40.0573,76.96,69,40,12.10,scattered clouds
657,Guarapari,BR,-20.6667,-40.4975,76.89,90,11,9.06,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.85,overcast clouds,-9.8000,-139.0333,
4,Kapaa,US,80.58,scattered clouds,22.0752,-159.3190,
5,Beibei,CN,81.07,broken clouds,29.8256,106.4339,
17,Busselton,AU,78.31,clear sky,-33.6500,115.3333,
26,Rikitea,PF,78.84,clear sky,-23.1203,-134.9692,
27,Vaini,TO,86.16,broken clouds,-21.2000,-175.2000,
33,Hilo,US,80.33,clear sky,19.7297,-155.0900,
37,Lorengau,PG,85.05,scattered clouds,-2.0226,147.2712,
38,Makakilo City,US,82.72,clear sky,21.3469,-158.0858,
39,Paciran,ID,84.42,overcast clouds,-6.8767,112.3761,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
 
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
  
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
       
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df = hotel_df.replace('', np.nan, regex=True)
hotel_df = hotel_df.dropna()
hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.85,overcast clouds,-9.8000,-139.0333,Villa Enata
4,Kapaa,US,80.58,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
5,Beibei,CN,81.07,broken clouds,29.8256,106.4339,FX Hotel Chongqing at Beibei Southwest University
17,Busselton,AU,78.31,clear sky,-33.6500,115.3333,Observatory Guest House
26,Rikitea,PF,78.84,clear sky,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
648,Bosaso,SO,75.56,overcast clouds,11.2842,49.1816,Red Sea Hotel
650,Byron Bay,AU,77.94,overcast clouds,-28.6500,153.6167,Backpackers Inn
651,Wajir,KE,76.96,scattered clouds,1.7471,40.0573,Al-bustan Hotel And Restaurant.
657,Guarapari,BR,76.89,few clouds,-20.6667,-40.4975,Hotel Porto do Sol


In [9]:
hotel_df.count()

City                   156
Country                156
Max Temp               156
Current Description    156
Lat                    156
Lng                    156
Hotel Name             156
dtype: int64

In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations  = hotel_df[['Lat', 'Lng']]
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Villa Enata</dd>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 78.85 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Sheraton Kauai Coconut Beach Resort</dd>\n<dt>City</dt><dd>Kapaa</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 80.58 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>FX Hotel Chongqing at Beibei Southwest University</dd>\n<dt>City</dt><dd>Beibei</dd>\n<dt>Country</dt><dd>CN</dd>\n<dt>Current Weather</dt><dd>broken clouds and 81.07 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Observatory Guest House</dd>\n<dt>City</dt><dd>Busselton</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>clear sky and 78.31 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>People ThankYou</dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>clear sky and 78.84 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hot

In [16]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=max_temp,
    dissipating=False, 
    max_intensity=300, 
    point_radius=4
)
marker_layer = gmaps.marker_layer(
    locations, 
    info_box_content=hotel_info
)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

AttributeError: module 'collections' has no attribute 'Iterable'